# Factorization Machines on MovieLens

### Download ml-100k dataset

In [ ]:
%%sh
wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
unzip ml-25m.zip

### Build training set and test set

In [ ]:
num_users=162541
num_movies=62423
num_ratings=25000095

max_movieid=209171

num_features=num_users+max_movieid

In [ ]:
print(num_features)

In [ ]:
import csv, sys
import numpy as np
from scipy.sparse import lil_matrix

def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter=',')
        next(samples)  # Skip header
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(num_users)+int(movieId)-1] = 1
            Y.append(float(rating))
            line=line+1
    Y=np.array(Y).astype('float32')
    return X,Y

In [ ]:
%%time
X, Y = loadDataset('ml-25m/ratings.csv', num_ratings, num_features)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05, random_state=59)

In [ ]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

### Convert to protobuf and save to S3

In [ ]:
import sagemaker, boto3

In [ ]:
bucket = sagemaker.Session().default_bucket()
prefix = 'fm-movielens-25m'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')
output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
%%time
import io
import sagemaker.amazon.common as smac

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

### Run training job

In [ ]:
# If you want to use existing files

train_data = 's3://sagemaker-us-east-1-613904931467/fm-movielens-25m/train/train.protobuf'
test_data  = 's3://sagemaker-us-east-1-613904931467/fm-movielens-25m/test/test.protobuf'

In [ ]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('factorization-machines', region)
print(container)

In [ ]:
fm = sagemaker.estimator.Estimator(container,
                                   role=sagemaker.get_execution_role(),
                                   instance_count=1, 
                                   instance_type='ml.c5.xlarge',
                                   output_path=output_prefix,
                                   volume_size=1
                                   )

fm.set_hyperparameters(feature_dim=num_features,
                      predictor_type='regressor',
                      num_factors=64,
                      epochs=1)

s3_train_data = sagemaker.TrainingInput(train_data, 
                                        distribution='FullyReplicated', 
                                        content_type='application/x-recordio-protobuf',
                                        s3_data_type='S3Prefix',
                                        input_mode='Pipe')

s3_test_data = sagemaker.TrainingInput(test_data,
                                             distribution='FullyReplicated', 
                                             content_type='application/x-recordio-protobuf', 
                                             s3_data_type='S3Prefix',
                                             input_mode='Pipe')
                                             
fm.fit({'train': s3_train_data, 'test': s3_test_data})

### Deploy model

In [ ]:
endpoint_name = 'fm-movielens-25m'
fm_predictor = fm.deploy(endpoint_name=endpoint_name,
                         instance_type='ml.t2.medium', initial_instance_count=1)

In [ ]:
import json

def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer

### Run predictions

In [ ]:
result = fm_predictor.predict(X_test[:3].toarray())
print(result)

In [ ]:
fm_predictor.delete_endpoint()